In [9]:
import pandas as pd
import numpy as np
import os, time,json,sys
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import random,warnings
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

In [10]:
warnings.simplefilter("ignore")
random.seed(42)
np.random.seed(42)
pd.set_option('display.max_columns', None)

In [11]:
train =pd.read_csv("DATA  Scientist Assignment/train.csv")
train_labels = pd.read_csv("DATA  Scientist Assignment/trainLabels.csv")
train_labels.drop("id", axis=1,inplace=True)
test_data = pd.read_csv("DATA  Scientist Assignment/test.csv")
test_data.columns = train.columns

In [12]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  #'mean', 'median', or 'most_frequent'
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

pca_pipe = Pipeline(steps=[
    ('PCA',PCA(n_components=25))
    ()
])

# bool_transformer = Pipeline(steps=[
#     ('imputer',KNNImputer())
# ])
# Bundle preprocessing for numerical and categorical features
preprocessor = ColumnTransformer(transformers=[
        ('num', numerical_transformer, make_column_selector(dtype_include=np.number)),
        ('cat', categorical_transformer, make_column_selector(dtype_include=object))
    ], remainder='passthrough',n_jobs=-1)


In [13]:
X = preprocessor.fit_transform(train)
y = train_labels[:9999]

In [14]:
# n_components=25
# pca = PCA(n_components=n_components)
# pca_output = pca.fit_transform(X.toarray())
# # pca_inverse = pca.inverse_transform(df)
# pca.get_covariance()

In [15]:
import xgboost as xgb
cls = MultiOutputClassifier(xgb.XGBClassifier(n_estimators=200, random_state=42,
                      objective='binary:logistic'))

In [16]:
# cls.fit(X, y)

In [17]:
X_train, X_val, y_train, X_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
# cls.fit(X, y)

In [19]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
X_train, X_val, y_train, X_val = train_test_split(X, y.values, test_size=0.2, random_state=42)
def objective(params):
#     test_model = xgb.XGBClassifier(objective='binary:logistic',random_state=42,verbose=1,early_stopping_rounds=5,verbose_eval=1,**params)
    test_model = xgb.XGBClassifier(objective='binary:logistic',random_state=42,
                                   verbose=1,**params)
#     test_model.fit(X_train, y_train,eval_set=[(X_val, y_val)])
    test_model.fit(X_train, y_train)

    #y_pred = test_model.predict(X_test)
    #print(y_pred.shape,y_test.shape)
    #regressor.score(X_test, y_test)
    neg_metric = -test_model.score(X_val, X_val)  # r2-score

    return {'loss': neg_metric, 'status': STATUS_OK}

In [20]:
search_space = {
    'max_depth': hp.choice('max_depth', range(3, 10)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
      'n_estimators': hp.choice('n_estimators', range(50, 530,30)),
    'reg_alpha': hp.uniform('reg_alpha', 0, 1),
      'reg_lambda': hp.uniform('reg_lambda', 0, 1)
}

In [21]:
trials = Trials()

# taking a lot of time but we can do this 

In [22]:
%%time
best_params = fmin(objective, search_space, algo=tpe.suggest, max_evals=100, trials=trials)

# Print the best hyperparameters
print(f"Best hyperparameters: {best_params}")

# Use the best hyperparameters to train the final model
model = xgb.XGBRegressor(**best_params)

[18:11:19] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627:                   
Parameters: { "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


  0%|                                                                          | 0/100 [00:00<?, ?trial/s, best loss=?]

job exception: Feature shape mismatch, expected: 34855, got 33



  0%|                                                                          | 0/100 [18:07<?, ?trial/s, best loss=?]


ValueError: Feature shape mismatch, expected: 34855, got 33